# Análisis Exploratorio y Preprocesamiento de Datos del Titanic

**Introducción:**

Este cuaderno presenta un análisis exploratorio de datos (EDA) y un proceso de preprocesamiento completo utilizando el clásico conjunto de datos del Titanic. El objetivo es transformar los datos crudos en un formato limpio, estructurado y enriquecido con nuevas características (feature engineering), preparándolo para ser utilizado en modelos de machine learning.

Se abordarán las siguientes etapas:
1.  **Configuración del Entorno:** Carga de librerías y montaje de Google Drive.
2.  **Carga de Datos:** Obtención del dataset del Titanic.
3.  **Análisis Exploratorio (EDA):** Comprensión inicial de la estructura, tipos de datos, valores faltantes y distribuciones de las variables.
4.  **Limpieza y Preprocesamiento:**
    * Tratamiento de valores nulos (imputación).
    * Ingeniería de características para crear nuevas variables predictivas.
5.  **Codificación de Variables Categóricas:** Aplicación de la técnica de **Weight of Evidence (WoE)** para transformar variables categóricas en numéricas, capturando su poder predictivo.
6.  **Generación del Dataset Final:** Consolidación de los datos preprocesados y guardado en un nuevo archivo.

## 1. Configuración del Entorno

En esta sección, se importan todas las librerías necesarias para el análisis y se configura el acceso a Google Drive para poder leer y escribir archivos.

In [1]:
# Importación de librerías estándar
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import requests
import os

# Instalar category_encoders si no se encuentra en el entorno
!pip install category_encoders -q
from category_encoders import WOEEncoder
from google.colab import drive

# Montaje de Google Drive
drive.mount("/content/drive")

# Creación de la carpeta de trabajo si no existe
contentFolder = "/content/drive/MyDrive/Austral - Data Mining"
if not os.path.exists(contentFolder):
    os.mkdir(contentFolder)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 2.4 MB/s eta 0:00:00
Mounted at /content/drive


## 2. Carga de Datos

Se descarga el archivo `titanic_train.txt` desde una URL pública y se carga en un DataFrame de Pandas para su posterior análisis.

In [2]:
# URL del archivo de entrenamiento del Titanic
url_titanic_train = "https://drive.google.com/uc?id=1hZAS7bZndWwIhpi7231sdQPfIfMhKyjb"
r = requests.get(url_titanic_train)
file_path = f"{contentFolder}/titanic_train.txt"
open(file_path, "wb").write(r.content)

# Carga del archivo en un DataFrame
data = pd.read_csv(file_path)

# Se establece PassengerId como índice para una mejor manipulación de los datos
data = data.set_index("PassengerId")

## 3. Análisis Exploratorio de Datos (EDA)

Antes de cualquier modificación, es fundamental entender la naturaleza de los datos. Se realizarán las siguientes verificaciones:
- **Diccionario de Datos:** Descripción de cada variable.
- **Inspección Inicial:** Visualización de las primeras y últimas filas.
- **Resumen Estadístico:** Métricas descriptivas para las variables numéricas.
- **Tipos de Datos y Nulos:** Identificación de los tipos de cada columna y el porcentaje de valores faltantes.

### Diccionario de Datos

- **Survived**: Sobrevivencia (0 = No, 1 = Sí) - **Variable Objetivo**
- **Pclass**: Clase del boleto (1 = 1ra, 2 = 2da, 3 = 3ra)
- **Sex**: Sexo (male, female)
- **Age**: Edad en años
- **SibSp**: Número de hermanos/cónyuges a bordo
- **Parch**: Número de padres/hijos a bordo
- **Ticket**: Número de boleto
- **Fare**: Tarifa del pasajero
- **Cabin**: Número de cabina
- **Embarked**: Puerto de embarque (C = Cherbourg, Q = Queenstown, S = Southampton)

In [3]:
# Visualización de las primeras 5 filas
print("Primeros registros del dataset:")
display(data.head())

# Resumen estadístico de las variables numéricas
print("\nResumen estadístico:")
display(data.describe())

# Información general del DataFrame: tipos de datos y nulos
print("\nInformación general y valores nulos:")
data.info()

Primeros registros del dataset:


,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S



Resumen estadístico:


,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200



Información general y valores nulos:
<class 'pandas.core.frame.DataFrame'>
Index: 891 entries, 1 to 891
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Name      891 non-null    object 
 3   Sex       891 non-null    object 
 4   Age       714 non-null    float64
 5   SibSp     891 non-null    int64  
 6   Parch     891 non-null    int64  
 7   Ticket    891 non-null    object 
 8   Fare      891 non-null    float64
 9   Cabin     204 non-null    object 
 10  Embarked  889 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 83.5+ KB


### Análisis de Valores Faltantes

Se calcula el porcentaje de valores nulos para cada columna. Esto es crucial para decidir la estrategia de imputación o eliminación de características.

- **Cabin:** Tiene un 77% de valores nulos. Es un candidato a ser eliminado.
- **Age:** Con casi un 20% de nulos, requiere una estrategia de imputación inteligente.
- **Embarked:** Tiene una cantidad mínima de nulos que pueden ser imputados fácilmente con la moda.

In [4]:
# Porcentaje de valores nulos por columna
null_percentage = (data.isnull().sum() / len(data) * 100).sort_values(ascending=False)
print("Porcentaje de valores nulos:")
print(null_percentage)

Porcentaje de valores nulos:
Cabin       77.104377
Age         19.865320
Embarked     0.224467
Name         0.000000
Pclass       0.000000
Survived     0.000000
Sex          0.000000
Parch        0.000000
SibSp        0.000000
Fare         0.000000
Ticket       0.000000
dtype: float64


## 4. Limpieza y Preprocesamiento

En esta fase, se aplican transformaciones a los datos para manejar los problemas identificados en el EDA.

In [5]:
# 1. Eliminación de la columna 'Cabin' por exceso de valores nulos
data = data.drop(columns=["Cabin"])

# 2. Imputación de 'Embarked' con la moda (el valor más frecuente)
embarked_mode = data["Embarked"].mode()[0]
data["Embarked"] = data["Embarked"].fillna(embarked_mode)

print(f"Valores nulos en 'Embarked' imputados con la moda: '{embarked_mode}'")

Valores nulos en 'Embarked' imputados con la moda: 'S'


### Imputación Inteligente de la Edad (`Age`)

La edad es una variable importante. En lugar de usar una simple media o mediana global, se utilizará una estrategia más sofisticada:
1.  **Extracción de Títulos:** Se extrae el título de cada pasajero (Mr, Miss, Mrs, etc.) de la columna `Name`.
2.  **Agrupación por Título:** Se calcula la edad promedio para cada título.
3.  **Imputación:** Se rellenan los valores nulos de `Age` con la edad promedio correspondiente a su título.

In [6]:
# 1. Extracción de títulos de la columna 'Name'
data['titulo'] = data['Name'].map(lambda x: x.split(',')[1].split('.')[0].strip())

# Visualización de la distribución de títulos
print("Distribución de títulos extraídos:")
print(data['titulo'].value_counts())

# 2. Cálculo de la edad promedio por título
titulo_edad_promedio = data.groupby('titulo')['Age'].mean().to_dict()

# 3. Imputación de nulos en 'Age' usando el promedio del título
data['Age'] = data.apply(
    lambda row: titulo_edad_promedio[row['titulo']] if pd.isnull(row['Age']) else row['Age'],
    axis=1
)

print(f"\nValores nulos restantes en 'Age': {data.Age.isnull().sum()}")

Distribución de títulos extraídos:
titulo
Mr              517
Miss            182
Mrs             125
Master           40
Dr                7
Rev               6
Col               2
Mlle              2
Major             2
Ms                1
Mme               1
Don               1
Lady              1
Sir               1
Capt              1
the Countess      1
Jonkheer          1
Name: count, dtype: int64

Valores nulos restantes en 'Age': 0


## 5. Ingeniería y Codificación de Características

Ahora que los datos están limpios, se crearán nuevas características y se codificarán las variables categóricas para que puedan ser interpretadas por un modelo.

### Weight of Evidence (WoE)

El **Peso de la Evidencia (WoE)** es una técnica de codificación de variables categóricas que mide la "fuerza" de una característica para separar eventos de no-eventos (en nuestro caso, `Survived = 1` vs `Survived = 0`).

La fórmula es:
$$ WoE = \ln\left(\frac{\% \text{ de no-eventos}}{\% \text{ de eventos}}\right) $$

- Un **WoE positivo** indica que la categoría tiene una mayor proporción de no-sobrevivientes.
- Un **WoE negativo** indica una mayor proporción de sobrevivientes.

Esta técnica es poderosa porque crea una relación monotónica con la variable objetivo, lo cual es beneficioso para modelos lineales como la regresión logística.

In [7]:
# Inicialización del codificador WoE
woe_encoder = WOEEncoder()

# Lista de columnas categóricas a codificar con WoE
cols_to_woe = ['titulo', 'SibSp', 'Parch']

for col in cols_to_woe:
    # Se ajusta y transforma la columna, asegurando que sea de tipo string
    woe_encoded = woe_encoder.fit_transform(data[col].astype(str), data.Survived)
    # Se añade la nueva columna codificada al DataFrame
    data[f'woe_{col}'] = woe_encoded

# Eliminación de las columnas originales y otras no necesarias para el modelo
data_final = data.drop(columns=['Name', 'Ticket', 'titulo', 'SibSp', 'Parch'])

print("Columnas después de la codificación WoE y limpieza:")
print(data_final.columns)

Columnas después de la codificación WoE y limpieza:
Index(['Survived', 'Pclass', 'Sex', 'Age', 'Fare', 'Embarked', 'woe_titulo',
       'woe_SibSp', 'woe_Parch'],
      dtype='object')


In [8]:
data_final.head()

,Survived,Pclass,Sex,Age,Fare,Embarked,woe_titulo,woe_SibSp,woe_Parch
PassengerId,,,,,,,,,
1,0,3,male,22.0,7.2500,S,-1.202121,0.613513,-0.173905
2,1,1,female,38.0,71.2833,C,1.780426,0.613513,-0.173905
3,1,3,female,26.0,7.9250,S,1.297772,-0.166010,-0.173905
4,1,1,female,35.0,53.1000,S,1.780426,0.613513,-0.173905
5,0,3,male,35.0,8.0500,S,-1.202121,-0.166010,-0.173905


### One-Hot Encoding

Para las variables categóricas restantes (`Sex` y `Embarked`), que tienen pocas categorías, se utilizará **One-Hot Encoding**. Esta técnica crea nuevas columnas binarias (0 o 1) para cada categoría.

In [9]:
# Aplicación de One-Hot Encoding a 'Sex' y 'Embarked'
data_final = pd.get_dummies(data_final, columns=['Sex', 'Embarked'], drop_first=True)

print("DataFrame final listo para modelado:")
display(data_final.head())

DataFrame final listo para modelado:


,Survived,Pclass,Age,Fare,woe_titulo,woe_SibSp,woe_Parch,Sex_male,Embarked_Q,Embarked_S
PassengerId,,,,,,,,,,
1,0,3,22.0,7.2500,-1.202121,0.613513,-0.173905,True,False,True
2,1,1,38.0,71.2833,1.780426,0.613513,-0.173905,False,False,False
3,1,3,26.0,7.9250,1.297772,-0.166010,-0.173905,False,False,True
4,1,1,35.0,53.1000,1.780426,0.613513,-0.173905,False,False,True
5,0,3,35.0,8.0500,-1.202121,-0.166010,-0.173905,True,False,True


## 6. Guardado del Dataset Preprocesado

Finalmente, se guarda el DataFrame procesado en un nuevo archivo CSV para su uso en futuros modelos de machine learning.

In [11]:
# Guardado del archivo preprocesado
preprocessed_path = f"{contentFolder}/titanic_preprocesado.csv"
data_final.to_csv(preprocessed_path)

print(f"Archivo preprocesado guardado en: {preprocessed_path}")

Archivo preprocesado guardado en: /content/drive/MyDrive/Austral - Data Mining/titanic_preprocesado.csv
